# Import libs and set configs

In [1]:
import requests as re
from bs4 import BeautifulSoup
import sqlite3
from datetime import date

import pandas as pd
import numpy as np

import time
from tqdm.notebook import tqdm

from scr.utils import get_user_profile, clear_link 
from scr.users_bgg import parse_loaded_users_info
from scr.loading_bgg import load_stack_of_users_to_buffer

class CFG:
    PATH_TO_DB = './data/board_games.sqlite'
    PATH_FOR_TMP_BG_FILES = './data/scraped/bgg_boardgame_tmp'
    PATH_FOR_TMP_USER_FILES = './data/scraped/bgg_user_tmp'

In [3]:
load_stack_of_users_to_buffer(PATH_TO_DB=CFG.PATH_TO_DB, PATH_TO_SAVE=CFG.PATH_FOR_TMP_USER_FILES, n_users=10)

100%|███████████████████████████████████████████| 10/10 [00:12<00:00,  1.26s/it]


In [15]:
if not [0]:
    print(1)
else:
    print(0)

0


In [17]:
not []

True

In [2]:
parse_loaded_users_info(PATH_TO_DB=CFG.PATH_TO_DB, PATH_TO_READ=CFG.PATH_FOR_TMP_USER_FILES, PATH_TO_SAVE=None)

  0%|                                                    | 0/11 [00:00<?, ?it/s]

add
add
add
add
add
add
add
add
add
add
add
add
add
add
add
add
add
add
add
add
add
add


 55%|████████████████████████                    | 6/11 [00:27<00:22,  4.54s/it]

add


IndexError: list index out of range

In [6]:
if []:
    print(True)
else: 
    print(False)

False


In [3]:
nicknames = get_users_for_scrap(CFG.PATH_TO_DB)

# User
add user into db by nickname

add columns to users table: https://boardgamegeek.com/user/zombiegod
- work link (boolean)
- Registration Date (date)
- Last Profile Update (date)
- Last Login (date)

aldoritm if we dont have user in db: 
- check link
- get profile info
- insert user into db

# User's collections
- Check 10k users with the oldest update date and download their data to disk
- Parse data and load into database


In [21]:
conn = sqlite3.connect(CFG.PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()

cursor.execute('''
            SELECT nickname
            FROM users
            ORDER BY last_check ASC
            LIMIt 10000
            ''')
data = cursor.fetchall()

# close the connection
conn.close()

In [23]:
nicknames = []
for i in data:
    nicknames.append(i[0])


In [5]:
conn = sqlite3.connect(CFG.PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()

cursor.execute('''
            SELECT boardgame_id
            FROM boardgames
            ''')
data = cursor.fetchall()

# close the connection
conn.close()

In [6]:
len(data)

134740

In [9]:
list_ids = []
for i in tqdm(data):
    list_ids.append(i[0])

  0%|          | 0/134740 [00:00<?, ?it/s]

In [4]:
get_api_user_ratings_data('AdrenRus', save_path=CFG.PATH_FOR_USER_FILES)

In [ ]:
get_api_bgg_game_data(unique_ids=[37111], save_path=CFG.PATH_FOR_BG_FILES)

In [3]:
add_user_into_db('AdrenRus', PATH_TO_DB=CFG.PATH_TO_DB)

In [4]:
conn = sqlite3.connect(CFG.PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()

cursor.execute('''
            SELECT *
            FROM users
            WHERE nickname = ?
            ''',('AdrenRus', ))
data = cursor.fetchall()
       

# cursor.execute('''
#             INSERT INTO descriptions (boardgame_id, description) VALUES (?, ?)
#             ''',(int(row.boardgame_id), row.description))

# commit the changes
# conn.commit()

# close the connection
conn.close()
data

[(1669691,
  'AdrenRus',
  None,
  None,
  '2021-11-08',
  '2022-12-19',
  '2023-04-13',
  '2023-04-13')]

In [33]:
user = get_user_profile('AdrenRus')
user

/Users/nikitafedorov/Documents/DataScience/Projects/Board-Game-Scraper/scr/utils.py:29: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 29 of the file /Users/nikitafedorov/Documents/DataScience/Projects/Board-Game-Scraper/scr/utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(r.text)


{'registration_date': '2021-11-08',
 'last_profile_update': '2022-12-19',
 'last_login': '2023-04-13',
 'country': '',
 'state': '',
 'city': '',
 'website': '',
 'geekmail': 'Send Private Message to AdrenRus'}

In [4]:
conn = sqlite3.connect(CFG.PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()

cursor.execute('''
            SELECT *
            FROM users
            WHERE nickname = ?
            ''',('AdrenRus', ))
data = cursor.fetchall()
       

# cursor.execute('''
#             INSERT INTO descriptions (boardgame_id, description) VALUES (?, ?)
#             ''',(int(row.boardgame_id), row.description))

# commit the changes
# conn.commit()

# close the connection
conn.close()
data

[]